<a href="https://colab.research.google.com/github/sciencebyAJ/OWRD_Training_20240927/blob/main/Example_1_OpenET_FlowMeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/sciencebyAJ/OWRD_Training_20240927.git

Cloning into 'OWRD_Training_20240927'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 83 (delta 24), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 754.77 KiB | 8.29 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
import geopandas as gpd
import glob

In [3]:
fields = gpd.read_file('/content/OWRD_Training_20240927/Shapefiles/flowmeter_shapefiles_v3/flowmeter_subset_fields_ORid_wgs84.shp')
fields_ss = fields[['OPENET_ID','Acres','SWU_ID','geometry']]

In [4]:
fields_ss.head()

,OPENET_ID,Acres,SWU_ID,geometry
0,ORx_24912,2.563825,SWU_UMAT0008,"POLYGON ((-118.44442 45.97865, -118.44443 45.9..."
1,ORx_24915,9.323391,SWU_UMAT0008,"POLYGON ((-118.44258 45.97911, -118.44187 45.9..."
2,ORx_25179,0.960043,SWU_UMAT0013,"POLYGON ((-118.38199 45.95786, -118.38197 45.9..."
3,ORx_25334,4.966444,SWU_UMAT0013,"POLYGON ((-118.38199 45.95744, -118.38187 45.9..."
4,ORx_25338,4.256518,SWU_UMAT0013,"POLYGON ((-118.38315 45.95842, -118.38204 45.9..."


In [5]:
hb = gpd.read_file('/content/OWRD_Training_20240927/Shapefiles/malheur_lake_basin_wgs84z11/malheur_lake_basin_wgs84z11.shp')

In [12]:
import ee
ee.Authenticate()
ee.Initialize(project='csumb-et-tools')
import geemap


In [23]:
fields_ss

,OPENET_ID,Acres,SWU_ID,geometry
0,ORx_24912,2.563825,SWU_UMAT0008,"POLYGON ((-118.44442 45.97865, -118.44443 45.9..."
1,ORx_24915,9.323391,SWU_UMAT0008,"POLYGON ((-118.44258 45.97911, -118.44187 45.9..."
2,ORx_25179,0.960043,SWU_UMAT0013,"POLYGON ((-118.38199 45.95786, -118.38197 45.9..."
3,ORx_25334,4.966444,SWU_UMAT0013,"POLYGON ((-118.38199 45.95744, -118.38187 45.9..."
4,ORx_25338,4.256518,SWU_UMAT0013,"POLYGON ((-118.38315 45.95842, -118.38204 45.9..."
5,ORx_25362,3.675433,SWU_UMAT0012,"POLYGON ((-118.39633 45.95753, -118.39645 45.9..."


In [35]:
SIMScollection = (
    ee.ImageCollection("OpenET/SIMS/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
SIMS = SIMScollection.select('et').sum()

EEMETRICcollection = (
    ee.ImageCollection("OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
EEMETRIC = EEMETRICcollection.select('et').sum()


ENScollection = (
    ee.ImageCollection("OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
ENS = ENScollection.select('et_ensemble_mad').sum()
ENS_range = ENScollection.select('et_ensemble_max').sum().subtract(ENScollection.select('et_ensemble_min').sum())


vis = {
    "min": 0.0,
    "max": 900,
    "palette": [
    '9e6212', 'ac7d1d', 'ba9829', 'c8b434', 'd6cf40', 'bed44b', '9fcb51',
    '80c256', '61b95c', '42b062', '45b677', '49bc8d', '4dc2a2', '51c8b8',
    '55cece', '4db4ba', '459aa7', '3d8094', '356681', '2d4c6e',]
}
m = geemap.Map(basemap="Esri.WorldImagery")

m.add_gdf(fields_ss, layer_name="WW Fields")
m.add_layer(SIMS, vis, "SIMS")
m.add_layer(EEMETRIC, vis, "EEMETRIC")
m.add_layer(ENS, vis, "ENS")
m.set_center(-118.424585, 45.970684, 14)

m


Map(center=[45.970684, -118.424585], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [ ]:

# m.add_gdf(hb, layer_name="Basin")
# m.set_center(-121.4181, 43.5790, 12)
